In [ ]:
#Install the foloowing packages if not done already

#!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 115.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.7/353.7 kB 38.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.3 MB/s eta 0:00:00


In [ ]:
# Download model artefacts. Gdown comes pre-installed with Colab. You may need to install gdown manually if you don't have it.
!gdown -q --folder 1A2nuQ1diz7AcoiKpyct7EKGaHtyrX7GD

In [36]:
import pandas as pd
from torch.utils.data import DataLoader
import numpy as np
import torch
from sentence_transformers import LoggingHandler, util
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CESoftmaxAccuracyEvaluator
from sentence_transformers.readers import InputExample
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import warnings
warnings.filterwarnings("ignore")

import logging
logging.disable(logging.WARNING)

In [55]:
#The models artefacts are present in Google Drive so this needs to be loaded

model_save_path = 'crossencoder_whitehouse_flangroberta_snli_sbert_with_claude'

num_labels = 2
model_ggea = CrossEncoder(model_save_path, num_labels=num_labels)

In [26]:
# Reference : https://huggingface.co/humarin/chatgpt_paraphraser_on_T5_base

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")

model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")#.to(device)

def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

In [100]:
#@title Enter Text which is to be paraphrased
text = "This year, our economy is projected to grow at the fastest pace in nearly 40 years. Right now, we have the opportunity to make once-in-a-generation investments in the foundations of middle class prosperity. Read more about the American Jobs Plan:" #@param {type:"string"}
#text = 'The American Jobs Plan is a once-in-a-generation investment in our infrastructure and future. Watch as President Biden sits down to share his thoughts about the plan:'

## Paraphraser

In [90]:
li_paraphrases = paraphrase(text)
li_paraphrases.append(text) # The original text can be the better than the paraphrases in generating retweets. Thus, we are adding the original text to the list of paraphrases.

responses = []
for i in li_paraphrases:
  print(i)

We are expecting the fastest growth in our economy for almost four decades this year. At present, we have the chance to invest in bolstering middle-class prosperity. Learn more about the American Jobs Plan:
The economy is expected to expand at a rate faster than it has been for almost 40 years this year. We have the opportunity to invest in the middle class's success once again. Learn more about the American Jobs Plan:
We are expecting the fastest growth in our economy for almost four decades this year. At present, we have the chance to invest in bolstering middle-class prosperity. Learn more about the American Jobs Plan: Why it matters?
The economy is expected to expand at a rate faster than it has been for almost 40 years this year. We have the opportunity to invest in the middle class's success once again. Learn more about the American Jobs Plan: Click here.
Our economy is poised to grow at its fastest rate in nearly 40 years this year, making it a prime opportunity for us all to in

# Getting responses from Anthropic Calude using API

In [ ]:
# If you have Anthropic key to access Calude lease mention here
ANTHROPIC_API_KEY = '<your key here>'
import anthropic

responses = []

def main(max_tokens_to_sample: int = 200):
    c = anthropic.Client(ANTHROPIC_API_KEY)
    for paraphrase in li_paraphrases:
      text = '''Why is the following text so engaging? \n Text: ''' + str(paraphrase)
      resp = c.completion(
          prompt= str(anthropic.HUMAN_PROMPT) + " " + str(text) + str(anthropic.AI_PROMPT),
          stop_sequences=[anthropic.HUMAN_PROMPT],
          model="claude-v1",
          max_tokens_to_sample=max_tokens_to_sample,
      )
      responses.append(resp['completion'])

if __name__ == "__main__":
    main()

# If you don't have access to Anthropic Claude, you can manually get the responses from Poe.com as mentioned below.

In [94]:
prefix = 'Why is the following text so engaging? \n Text: '
prompts = [prefix + str(i) for i in li_paraphrases]
for i in prompts:
  print(i)

Why is the following text so engaging? 
 Text: We are expecting the fastest growth in our economy for almost four decades this year. At present, we have the chance to invest in bolstering middle-class prosperity. Learn more about the American Jobs Plan:
Why is the following text so engaging? 
 Text: The economy is expected to expand at a rate faster than it has been for almost 40 years this year. We have the opportunity to invest in the middle class's success once again. Learn more about the American Jobs Plan:
Why is the following text so engaging? 
 Text: We are expecting the fastest growth in our economy for almost four decades this year. At present, we have the chance to invest in bolstering middle-class prosperity. Learn more about the American Jobs Plan: Why it matters?
Why is the following text so engaging? 
 Text: The economy is expected to expand at a rate faster than it has been for almost 40 years this year. We have the opportunity to invest in the middle class's success onc

In [ ]:
# Go to https://poe.com/Claude-instant, paste the prompts printed in the previous line one at a time, copy, and paste the responses below.

responses = ['''<response_1>''','''<response_2''','''<response_3>''','''<response_4''','''<response_5>''','''<response_6''']


# Example

"""

responses = ['''This text uses a few engaging rhetorical techniques:

1. It begins with optimistic framing: "We are expecting the fastest growth in our economy for almost four decades this year." Optimistic framing appeals to readers' hopes for economic progress and suggests positive change may be within reach. 

2. It presents an opportunity: "At present, we have the chance to invest in bolstering middle-class prosperity." Appealing to shared goals like middle-class prosperity and language of "chance" and "opportunity" can engage readers by giving them a sense of agency and possibility for change.

3. It provides a clear call to action at the end: "Learn more about the American Jobs Plan." Directives to take action give readers a sense of importance and motivation to be part of the solution.

4. The focus on goals like "middle-class prosperity" appeals to readers' hopes for economic fairness and opportunity - values that tend to resonate broadly. Appealing to shared values can engage diverse audiences.

5. The language is concise yet vivid, using simple words and active phrasing ("we have the chance"). Concise language tends to be more engaging for some readers, and active phrasing conveys a sense of agency.

However, the limited context could hinder engagement for some readers:

1. The text lacks details about the optimistic economic forecast and why growth is expected. Specifics would make the case more credible and compelling.

2. No details are provided about the "American Jobs Plan" or how it would bolster middle-class prosperity. Specifics would make the call to action more convincing.

3. The text avoids discussion of potential obstacles or trade-offs, which readers may see as an attempt to oversimplify complex issues. Addressing counterarguments could improve credibility.

In summary, the text employs some engaging techniques like optimistic framing, appealing to shared values and goals, and providing a call to action. But the lack of specifics about both the economic situation and the proposed solutions likely limits its ability to fully convince readers and inspire their action or support. Providing more details and context could enhance engagement.''',
'''This text uses some engaging rhetorical techniques:

1. It begins with optimistic framing: "The economy is expected to expand at a rate faster than it has been for almost 40 years this year." Optimistic framing appeals to readers' hopes for economic progress and suggests positive change may be within reach.

2. It presents an opportunity: "We have the opportunity to invest in the middle class's success once again." Appealing to shared goals like the middle class's success and using the language of "opportunity" can engage readers by giving them a sense of possibility for change.

3. It provides a clear call to action at the end: "Learn more about the American Jobs Plan." Directives to take action give readers a sense of importance and motivation to be part of the solution.     

4. The focus on supporting the middle class appeals to readers' hopes for economic fairness and opportunity - values that tend to resonate broadly. Appealing to shared values can potentially engage diverse audiences.

5. The language is concise, using simple words and active phrasing ("We have the opportunity"). Concise language tends to be more engaging for some readers.

However, the lack of details could limit engagement for some readers:

1. The text provides no specifics to support the optimistic economic forecast. Details would make the case more credible and compelling.

2. No details are given about the "American Jobs Plan" or how it would support the middle class. Specifics would make the call to action more convincing.

3. The text avoids discussing potential obstacles or trade-offs, which readers may see as an attempt to oversimplify complex issues. Addressing counterarguments could improve credibility.

In summary, the text employs some engaging techniques like optimistic framing, appealing to shared values, and providing a clear call to action. But the lack of specificity about both the economic situation and the proposed solutions likely limits its ability to fully convince readers and inspire their action or support. Providing more details and context could enhance engagement.''',
'''This text uses some engaging rhetorical techniques:

1. It begins with optimistic framing: "We are expecting the fastest growth in our economy for almost four decades this year." Optimistic framing appeals to readers' hopes for economic progress and suggests positive change may be within reach.

2. It presents an opportunity: "At present, we have the chance to invest in bolstering middle-class prosperity." Appealing to shared goals like middle-class prosperity and language of "chance" and "opportunity" can engage readers by giving them a sense of agency and possibility for change.

3. It provides a clear call to action: "Learn more about the American Jobs Plan: Why it matters?" Directives to take action give readers a sense of importance and motivation to be part of the solution.

4. The focus on goals like "middle-class prosperity" appeals to readers' hopes for economic fairness and opportunity - values that tend to resonate broadly. Appealing to shared values can engage diverse audiences. 

5. Adding "Why it matters?" indicates there may be details that justify the importance of the plan, which could help convince skeptical readers.

However, limitations remain that may reduce engagement for some readers:

1. The text lacks specifics to support the optimistic economic forecast, which would make the case more credible and compelling.

2. No details are provided about the "American Jobs Plan" or how it would bolster middle-class prosperity. Specifics would strengthen the call to action.

3. The text avoids discussing potential obstacles or trade-offs, which readers may see as an attempt to oversimplify complex issues. Addressing counterarguments could improve credibility.

In summary, while the text employs some engaging techniques, limitations like lack of specifics about both the economic situation and proposed solutions likely constrain its ability to fully convince readers and inspire their support. Providing additional details, justifications and context could enhance the text's credibility, persuasiveness and rhetorical engagement.

In short, details matter! The more specifics and evidence provided, the more convincing - and ultimately engaging - the text can be for readers.''',
'''This text uses some engaging rhetorical techniques:

1. It begins with optimistic framing: "The economy is expected to expand at a rate faster than it has been for almost 40 years this year." Optimistic framing appeals to readers' hopes for economic progress and suggests positive change may be within reach.

2. It presents an opportunity: "We have the opportunity to invest in the middle class's success once again." Appealing to shared goals like middle-class prosperity and language of "opportunity" can engage readers by giving them a sense of possibility for change.

3. It provides a clear call to action at the end: "Learn more about the American Jobs Plan: Click here." Directives to take action give readers a sense of importance and motivation to be part of the solution.   

4. The focus on supporting the middle class appeals to readers' hopes for economic fairness and opportunity - values that tend to resonate broadly. Appealing to shared values can potentially engage diverse audiences.

5. The language is concise, using simple words and active phrasing ("We have the opportunity"). Concise language tends to be more engaging for some readers.

However, the lack of details limits engagement for some readers:

1. The text provides no specifics to support the optimistic economic forecast. Details would make the case more credible and compelling.   

2. No details are given about the "American Jobs Plan" or how it would support the middle class. Specifics would strengthen the call to action.

3. The text avoids discussing potential obstacles or trade-offs, which readers may see as an attempt to oversimplify complex issues. Addressing counterarguments could improve credibility.

In summary, while the text employs some engaging techniques, the lack of specificity about both the economic situation and proposed solutions likely constrains its ability to fully convince readers and inspire their support. Providing more details, evidence and justifications could enhance the text's credibility, persuasiveness  and rhetorical engagement.

In short, the call to action may grab readers' attention initially, but without further specifics and context, the text's ability to fully engage and persuade readers remains limited.''',
'''This text uses a few potentially engaging rhetorical techniques:

1. It begins with optimistic framing: "Our economy is poised to grow at its fastest rate in nearly 40 years this year." Optimistic framing appeals to readers' hopes for economic progress and suggests positive change may be within reach.

2. It presents an opportunity: "making it a prime opportunity for us all to invest in the middle class's success." Appealing to shared goals like middle-class success and using the framing of "opportunity" can engage readers by giving them a sense of possibility for change.

3. It provides a call to action at the end: "Learn more about the American Jobs Plan." Directives to take action give readers a sense of importance and motivation to be part of the solution.

4. The focus on supporting the middle class appeals to readers' hopes for economic fairness and opportunity - values that tend to resonate broadly. Appealing to shared values can potentially engage diverse audiences.

5. The use of "us all" suggests inclusivity and a shared stake in the opportunity, which can engage readers by making them feel part of the collective "we."

However, the lack of details likely limits the text's engagement potential:

1. It provides no specifics to support the optimistic economic forecast. Details would make the case more credible and compelling.   

2. No details are given about the "American Jobs Plan" or how it would support the middle class. These specifics would strengthen the call to action.

3. The text avoids discussing potential obstacles or trade-offs, which readers may see as an attempt to oversimplify complex issues. Addressing counterarguments could improve credibility.

In summary, while the text employs some potentially engaging techniques, the lack of specificity about both the economic situation and proposed solutions likely constrains its ability to fully convince readers or inspire their support. Providing more details, evidence and justifications could enhance the text's credibility, persuasiveness and rhetorical engagement.

The initial optimism and call to action may grab readers' attention, but without further context, the text's ability to fully engage and persuade readers remains limited.''',
'''This text uses some engaging rhetorical techniques:

1. It begins with optimistic framing: "This year, our economy is projected to grow at the fastest pace in nearly 40 years." Optimistic framing appeals to readers' hopes for economic progress and suggests positive change may be within reach.

2. It presents an opportunity: "Right now, we have the opportunity to make once-in-a-generation investments in the foundations of middle class prosperity." Appealing to shared goals like middle class prosperity and using language of "opportunity" can engage readers by giving them a sense of agency and urgency.   

3. It provides a call to action at the end: "Read more about the American Jobs Plan." Directives to take action give readers a sense of importance and motivation.

4. The focus on "investments in the foundations of middle class prosperity" appeals to readers' hopes for economic fairness and opportunity - values that tend to resonate broadly. Appealing to shared values can engage diverse audiences.

5. Describing the investments as "once-in-a-generation" suggests an unprecedented and transformative opportunity, which can inspire and motivate readers.

However, limitations remain that may hinder engagement for some readers:

1. The text lacks specifics to support the optimistic economic forecast, which would make the case more credible and compelling.  

2. No details are provided about the "American Jobs Plan" or how it would achieve middle class prosperity. Such specifics would strengthen the call to action.

3. The text avoids discussing potential obstacles or trade-offs, which readers may see as an attempt to oversimplify complex issues. Addressing counterarguments could improve credibility.

In summary, though the text employs some potentially engaging techniques, limitations like lack of specifics about the economic situation and proposed solutions likely constrain its ability to fully convince readers or inspire their support. Providing more details, evidence and context could enhance the text's credibility, persuasiveness and rhetorical engagement.

The initial optimism and call to action may grab readers' attention, but without further context, the text's ability to fully engage and persuade readers remains limited.''']


"""

# Scoring GGEA

In [97]:
li_paraphrases_original_responses = [str(i) + " " + str(j) for i,j in zip(li_paraphrases, responses)]

In [98]:
di = {}
for i in range(0,len(li_paraphrases_original_responses)-1):
  for j in range(i+1,len(li_paraphrases_original_responses)):
    scores = model_ggea.predict([(li_paraphrases_original_responses[i], li_paraphrases_original_responses[j])])
    di[(i,j)] = [i if scores.argmax(axis=1)[0]==1 else j, scores[0].max()]

ind = list(di.values())[np.array([i[1] for i in di.values()]).argmax()][0]
print("Best paraphrased version is:\n", li_paraphrases[ind])    

Best paraphrased version is:
 The economy is expected to expand at a rate faster than it has been for almost 40 years this year. We have the opportunity to invest in the middle class's success once again. Learn more about the American Jobs Plan:
